# Bab 12: Custom Models and Training with TensorFlow

### 1. Pendahuluan

Bab 12 membawa kita lebih dalam ke dunia TensorFlow, melampaui API tingkat tinggi Keras yang telah kita gunakan sejauh ini. Meskipun Keras sangat kuat dan mudah digunakan, terkadang kita memerlukan fleksibilitas lebih untuk membangun arsitektur yang tidak biasa, membuat *loss function* kustom, atau bahkan menulis *training loop* sendiri. Bab ini akan menunjukkan bagaimana cara bekerja dengan API tingkat rendah TensorFlow.

Topik yang akan dibahas meliputi:
* **Dasar-dasar TensorFlow:** Bekerja dengan *tensors* dan operasi-operasi dasar.
* **Kustomisasi:**
    * Membuat *loss function* dan metrik kustom.
    * Membuat *layer* dan model kustom dengan *subclassing*.
    * Menulis *training loop* kustom dari awal.
* **Performa:** Bagaimana TensorFlow mengubah fungsi Python menjadi graf komputasi berperforma tinggi menggunakan `tf.function`.

---

### 2. Sekilas tentang TensorFlow

TensorFlow adalah *library* yang kuat untuk komputasi numerik, khususnya untuk *machine learning* skala besar. Unit data fundamental di TensorFlow adalah **tensor**, yang sangat mirip dengan NumPy `ndarray`.


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# Membuat Tensor
t = tf.constant([[1., 2., 3.], [4., 5., 6.]]) # matrix
print("Tensor t:\n", t)

# Indexing
print("\nIndexing t[:, 1:]:", t[:, 1:])

# Operasi Tensor
print("\nt + 10:", t + 10)
print("tf.square(t):", tf.square(t))

# Tensor dan NumPy
# Konversi dari NumPy array ke Tensor
a = np.array([2., 4., 5.])
b = tf.constant(a)
print("\nTensor dari NumPy:", b)

# Konversi dari Tensor ke NumPy array
print("NumPy dari Tensor:", t.numpy())

Tensor t:
 tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)

Indexing t[:, 1:]: tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)

t + 10: tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.square(t): tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)

Tensor dari NumPy: tf.Tensor([2. 4. 5.], shape=(3,), dtype=float64)
NumPy dari Tensor: [[1. 2. 3.]
 [4. 5. 6.]]


---

### 3. Kustomisasi Model dan Algoritma Pelatihan
Walaupun Keras menyediakan banyak komponen siap pakai, fleksibilitas TensorFlow memungkinkan kita untuk mengkustomisasi hampir setiap bagian dari model.

### a. Custom Loss Functions
Membuat loss function kustom di Keras sangatlah mudah. Cukup buat sebuah fungsi yang menerima y_true (label sebenarnya) dan y_pred (prediksi) sebagai argumen, lalu gunakan operasi TensorFlow untuk menghitung loss.

Berikut adalah contoh implementasi Huber loss, yang tidak sensitif terhadap outlier besar.
$$ \text{Huber}(y, \hat{y}, \delta) =
\begin{cases}
\frac{1}{2}(y - \hat{y})^2 & \text{if } |y - \hat{y}| \le \delta \
\delta(|y - \hat{y}| - \frac{1}{2}\delta) & \text{otherwise}
\end{cases}
$$

In [2]:
# Fungsi Huber loss kustom
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

# Contoh penggunaan pada model Keras
# model.compile(loss=huber_fn, optimizer="nadam")
# model.fit(X_train, y_train, ...)

### b. Custom Layers
Untuk arsitektur yang eksotis atau layer yang tidak biasa, Anda dapat membuat layer kustom sendiri dengan membuat subclass dari keras.layers.Layer.

In [3]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

### c. Custom Models
Mirip dengan layer, Anda dapat membuat model dengan arsitektur yang sangat fleksibel dengan membuat subclass dari keras.Model.

In [4]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        # ... implementasi block (misalnya, untuk ResNet)
        pass
    def call(self, inputs):
        # ...
        pass

class ResidualRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        # Definisikan layer-layer di sini
        self.hidden1 = keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        # Definisikan alur forward pass di sini
        Z = self.hidden1(inputs)
        for _ in range(1, 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

---

### 4. Custom Training Loops
Menulis training loop kustom memberikan kontrol penuh atas proses pelatihan. Ini berguna untuk penelitian atau saat ingin mencoba ide-ide baru yang tidak didukung oleh metode .fit() bawaan.

Prosesnya secara umum adalah:

1. Buat nested loop: satu untuk epoch, satu lagi untuk batch di dalam setiap epoch.
2. Di dalam loop batch, buat tf.GradientTape() context.
3. Lakukan forward pass di dalam context tersebut untuk membuat prediksi.
4. Hitung loss berdasarkan prediksi dan label sebenarnya.
5. Gunakan tape untuk menghitung gradien dari loss terhadap setiap trainable variable model.
6. Terapkan gradien ini pada variable menggunakan optimizer untuk memperbarui bobot.
7. Perbarui metrik, cetak status, dan ulangi.

In [5]:
# (Asumsikan model, optimizer, loss_fn, dan dataset sudah didefinisikan)

n_epochs = 5
batch_size = 32
# dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train)).batch(batch_size)
# mean_loss = keras.metrics.Mean()
# metrics = [keras.metrics.MeanAbsoluteError()]

# for epoch in range(1, n_epochs + 1):
#     print(f"Epoch {epoch}/{n_epochs}")
#     for X_batch, y_batch in dataset:
#         with tf.GradientTape() as tape:
#             y_pred = model(X_batch, training=True)
#             main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
#             loss = tf.add_n([main_loss] + model.losses)
        
#         gradients = tape.gradient(loss, model.trainable_variables)
#         optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
#         # Update metrics
#         mean_loss(loss)
#         for metric in metrics:
#             metric(y_batch, y_pred)
#     print(f"  loss: {mean_loss.result():.4f}", end="")
#     for metric in metrics:
#         print(f" - {metric.name}: {metric.result():.4f}", end="")
#     print()

---

### 5. TensorFlow Functions dan Graphs
Performa Python tidak secepat C++. Untuk mengatasinya, TensorFlow dapat mengubah fungsi Python menjadi graf komputasi berperforma tinggi. Ini dilakukan dengan membungkus fungsi tersebut dalam tf.function.

In [6]:
# Fungsi Python biasa
def cubed(x):
    return x ** 3

# Diubah menjadi TensorFlow Function
tf_cubed = tf.function(cubed)

# Atau menggunakan decorator
@tf.function
def tf_cubed_decorated(x):
    return x ** 3

Saat Anda memanggil tf.function untuk pertama kalinya, ia akan "menelusuri" (trace) fungsi tersebut, menganalisis semua operasi TensorFlow, dan membangun graf komputasi yang setara. Setelah itu, TensorFlow akan mengeksekusi graf yang sudah dioptimalkan ini, yang jauh lebih cepat daripada mengeksekusi kode Python baris per baris. Keras secara otomatis melakukan ini untuk model Anda saat Anda memanggil .fit().

---

### 6. Kesimpulan
Bab ini menunjukkan kekuatan dan fleksibilitas TensorFlow di luar API tingkat tinggi Keras. Dengan kemampuan untuk membuat layer, loss, model, dan bahkan training loop kustom, Anda memiliki alat yang diperlukan untuk mengimplementasikan hampir semua arsitektur deep learning, bahkan yang paling baru sekalipun. Memahami tf.function juga penting untuk memastikan kode kustom Anda berjalan seefisien mungkin.